In [3]:
import random
import json

import numpy as np

import pandas as pd

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator

import math

In [4]:
class WordleCritic:
    """
    Klasse, die ein Wordle Spiel repräsentiert
    Bekommt ein Startwort zugewiesen und bewertet Vorschläge im Stile eines Wordle Spiels
    """
        
    def __init__(self, word):
        """
        Parameters
        ----------
        word: string
            Das zu erratende Wort
        """
        self.word = word
        
    """
    Bewertet einen Vorschlag und gibt das Ergebnis im Stile eines Wordle-Spiels zurück (grau, gelb, grün)
    
    Parameters
    ----------
    suggestion: string
        Der Vorschlag, der bewertet werden soll
    """
    def judge(self, suggestion):
        result = ["gray" for i in range(len(self.word))]
        
        for index, letter in enumerate(suggestion):
            if letter == self.word[index]:
                result[index] = "green"
            elif letter in self.word:
                result[index] = "yellow"
                
        return result

In [5]:
class WordlePlayer:
    """
    Klasse, die einen Wordle-Spieler repräsentiert
    Generiert einen möglichen nächsten Vorschlag, basierend auf dem bereits bekannten Informationen
    Die Vorschläge werden auf Basis eines Bayes'schen Netzes mit der Bibliothek pgmpy generiert
    """
    
    def __init__(self, words_file, model):
        """
        Parameters
        ----------
        words_file: string
            Name der Wortliste (selber Name für CSV und JSON Datei)
        model: BayesianNetwork
            Das Bayes'sche Netzwerk von pgmpy
        """
        
        self.words_pd = pd.read_csv(words_file + ".csv")
        
        with open(words_file + ".json", "r") as f:
            self.words_list = json.load(f) 
        
        self.model = model

        self.model.fit(self.words_pd, estimator=MaximumLikelihoodEstimator)
        self.infer = VariableElimination(self.model)
        
    """
    Setzt ein Wort aus den bereits bekannten Informationen und der Rückgabe der Veriable Elimination zusammen
    
    Parameters
    ----------
    suggestion: dict
        Das Ergebnis der Variable Elimination auf dem Bayes'schen Netz
    evidence: dict
        Die bereits bekannten Zustände
    """
    def get_suggestion_word(self, suggestion, evidence):
        word = ["", "", "", "", ""]

        if "first" in evidence:
            word[0] = evidence["first"]
        else: 
            word[0] = suggestion["first"]

        if "second" in evidence:
            word[1] = evidence["second"]
        else:
            word[1] = suggestion["second"]

        if "third" in evidence:
            word[2] = evidence["third"]
        else:
            word[2] = suggestion["third"]

        if "forth" in evidence:
            word[3] = evidence["forth"]
        else:
            word[3] = suggestion["forth"]

        if "fifth" in evidence:
            word[4] = evidence["fifth"]
        else:
            word[4] = suggestion["fifth"]

        return word

    """
    Überprüft, ob ein Wort ein valider Vorschlag ist
    
    Parameters
    ----------
    must_contain: list, optional
        Die Buchstaben, welche im Wort enthalten sein müssen (gelb markiert)
    must_not_contain: list, optional
        Die Buchstaben, welche nicht im Wort enthalten sein dürfen (grau markiert)
    must_not_contain_at: dict, optional
        Jeweils fünf Listen, mit den Buchstaben die an der jeweiligen Position nicht vorhanden sein dürfen 
        (z.B. {"first": ["A", "B"], "second": ["S"], "third": [], "forth": [], "fifth": []})
    """
    def word_is_valid(self, word, must_contain=[], must_not_contain=[], must_not_contain_at={}):
        if "".join(str(char) for char in word) in self.words_list:

            for letter in must_contain:
                if not letter in word:
                    return False

            for letter in must_not_contain:
                if letter in word:
                    return False

            if "first" in must_not_contain_at and word[0] in must_not_contain_at["first"]:
                return False
            
            if "second" in must_not_contain_at and word[1] in must_not_contain_at["second"]:
                return False

            if "third" in must_not_contain_at and word[2] in must_not_contain_at["third"]:
                return False

            if "forth" in must_not_contain_at and word[3] in must_not_contain_at["forth"]:
                return False

            if "fifth" in must_not_contain_at and word[4] in must_not_contain_at["fifth"]:
                return False

            return True
        else:
            return False

    """
    Genereriert einen neuen Vorschlag, basierend auf: 
    
    Parameters
    ----------
    variables: list
        Die Variablen an denen die Buchstaben noch unbekannt und gesucht sind (z.B. ["first", "second"])
    evidence: dict
        Die bereits bekannten Variablen und deren Zustand (z.B. {"first": "A", "second": "S"})
    must_contain: list, optional
        Die Buchstaben, welche im Wort enthalten sein müssen (gelb markiert)
    must_not_contain: list, optional
        Die Buchstaben, welche nicht im Wort enthalten sein dürfen (grau markiert)
    must_not_contain_at: dict, optional
        Jeweils fünf Listen, mit den Buchstaben die an der jeweiligen Position nicht vorhanden sein dürfen 
        (z.B. {"first": ["A", "B"], "second": ["S"], "third": [], "forth": [], "fifth": []})
    """
    def get_suggestion(self, variables, evidence, must_contain=[], must_not_contain=[], must_not_contain_at=[]):
        q = self.infer.query(variables, evidence=evidence, show_progress=False)

        count_predictions = len(q.values.flatten()[q.values.flatten() != 0])
        max_value_indices = (-q.values.flatten()).argsort()[:count_predictions]

        result = []

        for max_value_index in max_value_indices:
            indices = np.unravel_index(max_value_index, q.values.shape)

            suggestion = {}

            for index, variable in enumerate(q.variables):
                suggestion[variable] = self.model.get_cpds(variable).state_names[variable][indices[index]]

            word = self.get_suggestion_word(suggestion, evidence)

            if self.word_is_valid(word, must_contain, must_not_contain, must_not_contain_at):
                #result.append(word)
                return word

        return result
    
    """Generiert den ersten Vorschlag"""
    def get_first_suggestion(self):
        first = self.infer.map_query(["first"], show_progress=False)["first"]
        second = self.infer.map_query(["second"], evidence={"first": first}, show_progress=False)["second"]
        third = self.infer.map_query(["third"], evidence={"first": first, "second": second}, show_progress=False)["third"]
        forth = self.infer.map_query(["forth"], evidence={"first": first, "second": second, "third": third}, show_progress=False)["forth"]
        fifth = self.infer.map_query(["fifth"], evidence={"first": first, "second": second, "third": third, "forth": forth}, show_progress=False)["fifth"]

        if self.word_is_valid("".join([first,second,third,forth,fifth])):
            return "".join[first,second,third,forth,fifth]
        elif len(self.words_pd[
                (self.words_pd["first"] == first) & 
                (self.words_pd["second"] == second) & 
                (self.words_pd["third"] == third) & 
                (self.words_pd["forth"] == forth)]) > 0:
            return self.words_pd[
                (self.words_pd["first"] == first) & 
                (self.words_pd["second"] == second) & 
                (self.words_pd["third"] == third) & 
                (self.words_pd["forth"] == forth)].values[0]
        else:
            return self.words_pd[
                (self.words_pd["first"] == first) & 
                (self.words_pd["second"] == second) & 
                (self.words_pd["third"] == third)].values[0]
        

In [6]:
"""
Neues Wordle-Spiel mit einem zufälligen Wort aus der Wortliste
"""

with open("words.json", "r") as f:
    words = json.load(f)

word = random.choice(words)

critic = WordleCritic(word)

print(critic.word)

FLIRT


In [8]:
"""
Selbstspielendes Wordle-Spiel
Das erstellte Player-Objekt verarbeitet die Informationen, die der Critic liefert und generiert so lange neue 
Vorschläge, bis das zu erratende Wort gefunden ist
"""

model = BayesianNetwork([
    ("first", "second"), 
    ("third", "second"), 
    ("third", "forth"), 
    ("fifth", "forth")
])

player = WordlePlayer("words", model)

suggestion = player.get_first_suggestion()

known_information = {
    "variables":set([]), 
    "evidence":{}, 
    "must_contain":set([]), 
    "must_not_contain":set([]),
    "must_not_contain_at":{
        "first":set([]), 
        "second":set([]), 
        "third":set([]),
        "forth":set([]),
        "fifth":set([])
    }
}

tries = 0

while True:
    evidences = critic.judge(suggestion)
    tries += 1
    
    print(str(tries) + ": " + "".join(suggestion))
    print(evidences)
    print()
    
    if evidences[0] == "green" and evidences[1] == "green" and evidences[2] == "green" and evidences[3] == "green" and evidences[4] == "green":
        break
    
    if evidences[0] == "gray":
        known_information["variables"].add("first")
        known_information["must_not_contain"].add(suggestion[0])
    elif evidences[0] == "yellow":
        known_information["variables"].add("first")
        known_information["must_contain"].add(suggestion[0])
        known_information["must_not_contain_at"]["first"].add(suggestion[0])
    elif evidences[0] == "green":
        known_information["variables"].discard("first")
        known_information["evidence"]["first"] = suggestion[0]

    if evidences[1] == "gray":
        known_information["variables"].add("second")
        known_information["must_not_contain"].add(suggestion[1])
    elif evidences[1] == "yellow":
        known_information["variables"].add("second")
        known_information["must_contain"].add(suggestion[1])
        known_information["must_not_contain_at"]["second"].add(suggestion[1])
    elif evidences[1] == "green":
        known_information["variables"].discard("second")
        known_information["evidence"]["second"] = suggestion[1]

    if evidences[2] == "gray":
        known_information["variables"].add("third")
        known_information["must_not_contain"].add(suggestion[2])
    elif evidences[2] == "yellow":
        known_information["variables"].add("third")
        known_information["must_contain"].add(suggestion[2])
        known_information["must_not_contain_at"]["third"].add(suggestion[2])
    elif evidences[2] == "green":
        known_information["variables"].discard("third")
        known_information["evidence"]["third"] = suggestion[2]

    if evidences[3] == "gray":
        known_information["variables"].add("forth")
        known_information["must_not_contain"].add(suggestion[3])
    elif evidences[3] == "yellow":
        known_information["variables"].add("forth")
        known_information["must_contain"].add(suggestion[3])
        known_information["must_not_contain_at"]["forth"].add(suggestion[3])
    elif evidences[3] == "green":
        known_information["variables"].discard("forth")
        known_information["evidence"]["forth"] = suggestion[3]

    if evidences[4] == "gray":
        known_information["variables"].add("fifth")
        known_information["must_not_contain"].add(suggestion[4])
    elif evidences[4] == "yellow":
        known_information["variables"].add("fifth")
        known_information["must_contain"].add(suggestion[4])
        known_information["must_not_contain_at"]["fifth"].add(suggestion[4])
    elif evidences[4] == "green":
        known_information["variables"].discard("fifth")
        known_information["evidence"]["fifth"] = suggestion[4]

    suggestion = player.get_suggestion(
        known_information["variables"], 
        known_information["evidence"],
        known_information["must_contain"],
        known_information["must_not_contain"],
        known_information["must_not_contain_at"]
    )
    
print("The word you are looking for is " + "".join(suggestion) + "!")
print("Found it in " + str(tries) + " tries!")

1: SABOR
['gray', 'gray', 'gray', 'gray', 'yellow']

2: REINE
['yellow', 'gray', 'green', 'gray', 'gray']

3: TRIFT
['yellow', 'yellow', 'green', 'yellow', 'green']

4: FLIRT
['green', 'green', 'green', 'green', 'green']

The word you are looking for is FLIRT!
Found it in 4 tries!
